In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

# Any results you write to the current directory are saved as output.

['sample_submission_stage_1.csv', 'test_stage_1.tsv', 'trees']


In [1]:
import pickle

In [3]:
test_feats = pickle.load(open('dataset/multi_inputs/test_feats.pkl', 'rb'))
dev_feats = pickle.load(open('dataset/multi_inputs/dev_feats.pkl', 'rb'))
val_feats = pickle.load(open('dataset/multi_inputs/val_feats.pkl', 'rb'))

In [70]:
dev_feats[0]['a'].keys()

dict_keys(['all', 'sum', 'mean', 'max'])

In [36]:
import numpy as np

In [46]:
def generate_train_test_set(focus_on='max'):
    
    def prepare_data(feats, focus_on):
        x, y = [], []
        for feat in feats:
            x.append(np.concatenate([feat['a'][focus_on], feat['b'][focus_on], feat['p']], axis=0))
            y.append(feat['label'])
        return x, y
            
    test_x, test_y = prepare_data(test_feats, focus_on)
    val_x, val_y = prepare_data(val_feats, focus_on)
    dev_x, dev_y = prepare_data(dev_feats, focus_on)
    
    train_x, train_y = np.concatenate([test_x, val_x], axis=0), np.concatenate([test_y, val_y], axis=0)
    return (train_x, train_y), (dev_x, dev_y)

In [77]:
(train_x, train_y), (dev_x, dev_y) = generate_train_test_set('mean')

In [78]:
from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time


dense_layer_sizes = [37]
dropout_rate = 0.6
learning_rate = 0.001
n_fold = 5
batch_size = 32
epochs = 1000
patience = 100
# n_test = 100
lambd = 0.1 # L2 regularization

In [79]:
def build_mlp_model(input_shape):
	X_input = layers.Input(input_shape)
	X = layers.Dropout(dropout_rate, seed = 7)(X_input)
	# First dense layer
	X = layers.Dense(dense_layer_sizes[0], name = 'dense0')(X)
	X = layers.BatchNormalization(name = 'bn0')(X)
	X = layers.Activation('relu')(X)
	X = layers.Dropout(dropout_rate, seed = 7)(X)

	# Output layer
	X = layers.Dense(3, name = 'output', kernel_regularizer = regularizers.l2(lambd))(X)
	X = layers.Activation('softmax')(X)

	# Create model
	model = models.Model(input = X_input, output = X, name = "classif_model")
	return model

In [80]:
from keras.utils import to_categorical
X_train, Y_train = train_x, to_categorical(train_y)
X_development, Y_development = np.array(dev_x), to_categorical(dev_y)

In [84]:
# Training and cross-validation
test_num = 4
for run in range(test_num):
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=3)
    scores = []
    oof = np.zeros_like(Y_train)
    prediction = np.zeros_like(Y_development)

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train)):
        # split training and validation data
        print('Fold', fold_n, 'started at', time.ctime())
        X_tr, X_val = X_train[train_index], X_train[valid_index]
        Y_tr, Y_val = Y_train[train_index], Y_train[valid_index]

        # Define the model, re-initializing for each fold
        classif_model = build_mlp_model([X_train.shape[1]])
        classif_model.compile(optimizer = optimizers.Adam(lr = learning_rate), loss = "categorical_crossentropy")
        callbacks = [kc.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights = True)]

        # train the model
        classif_model.fit(x = X_tr, y = Y_tr, epochs = epochs, batch_size = batch_size, callbacks = callbacks, validation_data = (X_val, Y_val), verbose = 0)

        # make predictions on validation and test data
        pred_valid = classif_model.predict(x = X_val, verbose = 0)
        oof[valid_index] = pred_valid
        pred = classif_model.predict(x = X_development, verbose = 0)

        # oof[valid_index] = pred_valid.reshape(-1,)
        scores.append(log_loss(Y_val, pred_valid))
        prediction += pred

    prediction /= n_fold

    # Print CV scores, as well as score on the test data
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    print(scores)
    print("Test score:", log_loss(Y_development, prediction))

Fold 0 started at Sat Apr 13 03:20:06 2019


C:\Users\zake7\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classif_model", inputs=Tensor("in..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


Fold 1 started at Sat Apr 13 03:24:04 2019
Fold 2 started at Sat Apr 13 03:31:19 2019
Fold 3 started at Sat Apr 13 03:36:25 2019
Fold 4 started at Sat Apr 13 03:41:24 2019
CV mean score: 0.5234, std: 0.0447.
[0.48127582803266145, 0.46048753476395704, 0.5626410425670221, 0.5725375277047746, 0.5402216598068049]
Test score: 0.4815043754691924


# Max Pooling

In [ ]:
CV mean score: 0.5285, std: 0.0338.
[0.4952458826401328, 0.48375502069637916, 0.5682628710749237, 0.5597365417256113, 0.5356817798765776]
Test score: 0.49103962449882965
    
CV mean score: 0.5243, std: 0.0379.
[0.5003560718592084, 0.4685565223567941, 0.5682392502662073, 0.563904224778733, 0.5206329654054942]
Test score: 0.49357280478751636
    
CV mean score: 0.5240, std: 0.0383.
[0.4950065191369652, 0.4670000282987749, 0.5668030280902658, 0.5613617074133851, 0.5296100813049671]
Test score: 0.4927514025819255
    
CV mean score: 0.5263, std: 0.0378.
[0.49447658181332393, 0.472486299146505, 0.5725461843181943, 0.5589639414076943, 0.5332190037439506]
Test score: 0.4944068343992476    

# Mean Pooling

In [ ]:
CV mean score: 0.5238, std: 0.0468.
[0.47688187236489304, 0.46213150363347727, 0.5698529141631439, 0.5761041223376069, 0.5340085791760689]
Test score: 0.48395181742742716
    
CV mean score: 0.5225, std: 0.0410.
[0.48435351767456547, 0.46383921129004496, 0.5586745387855813, 0.5662293704637682, 0.5392278727089953]
Test score: 0.48673919674764327
    
CV mean score: 0.5280, std: 0.0457.
[0.47838048707321157, 0.4697366463747121, 0.5715442865168441, 0.5776206567589667, 0.542755629628534]
Test score: 0.47883237509357784  
    
CV mean score: 0.5234, std: 0.0447.
[0.48127582803266145, 0.46048753476395704, 0.5626410425670221, 0.5725375277047746, 0.5402216598068049]
Test score: 0.4815043754691924        

# Sum Pooling

In [ ]:
CV mean score: 0.5315, std: 0.0388.
[0.48817598121504135, 0.48436255429821085, 0.5787604555612169, 0.5645790173302089, 0.5416744311974758]
Test score: 0.50545762531316

CV mean score: 0.5403, std: 0.0321.
[0.4971414440332669, 0.5091939038344715, 0.5813902868192085, 0.5635956079994316, 0.5503828079077386]
Test score: 0.5067319352220293
    
CV mean score: 0.5375, std: 0.0362.
[0.4896285316412134, 0.49989288843345275, 0.5773998257683622, 0.5704335607287065, 0.5503486714648511]
Test score: 0.5037706942331842

CV mean score: 0.5369, std: 0.0376.
[0.49427899776592293, 0.49054143951267815, 0.5830774302662529, 0.5633550574142361, 0.5533396635592407]
Test score: 0.5078974904599308    